In [115]:
import openai
import requests
import json
import math
import os
from dotenv import load_dotenv
from openai.types.chat import ChatCompletionToolParam, ChatCompletionToolMessageParam, ChatCompletionUserMessageParam

In [99]:
load_dotenv()
openai.api_key = os.getenv('API_KEY')
open_weather_api_key = os.getenv('OPEN_WEATHER_API_KEY')
model = "gpt-3.5-turbo-0613" # 0613以降のバージョンでないとFunction callingは機能しません


In [100]:
# OpenWeather で指定の都市の天気を取得する
def get_current_weather(city_name="Tokyo"):
    # 都市の緯度、経度を取得する
    geocoding_response = requests.get("http://api.openweathermap.org/geo/1.0/direct", params={
        "q": city_name + ",jp",
        "limit": 1,
        "appid": open_weather_api_key
    })
    geocodings = geocoding_response.json()
    geocoding = geocodings[0]
    #print(geocoding)
    lat, lon = geocoding["lat"], geocoding["lon"]

    # 指定した緯度、経度の現在の天気を取得する
    current_weather_response = requests.get("https://api.openweathermap.org/data/2.5/weather", params={
        "lat": lat,
        "lon": lon,
        "units": "metric",
        "lang": "ja",
        "appid": open_weather_api_key
    })
    current_weather = current_weather_response.json()
    #print(current_weather)

    return {
        "city_name": city_name,
        "description": current_weather["weather"][0]["description"],
        "temparature": math.floor(current_weather["main"]["temp"])
    }

In [101]:
# 現在時刻を教えてくれる関数
from datetime import datetime

def get_current_datetime():
    now = datetime.now()
    current_date = now.strftime("%Y-%m-%d") 
    return {
        "now_time": current_date
    }

In [102]:
test_city = get_current_weather("大阪")
test_sports = get_current_datetime()
print(test_city)
print(test_sports)

{'city_name': '大阪', 'description': '曇りがち', 'temparature': 12}
{'now_time': '2024-02-17'}


In [103]:
# function callingで利用する関数
functions = [
    # dictで関数を定義
    {
        # 外部関数の名前
        "name": "get_current_weather",
        # 外部関数の説明
        "description": "指定された都市の現在の天気を取得する",
        
        "parameters": {
            "type": "object",
            # 外部関数の引数 = レスポンスに含まれるフィールド
            "properties": {
                "city_name": {
                    "type": "string",
                    "description": "英語表記の都市名",
                },
            },
            # 必須のフィールド
            "required": ["city_name"],
        },
    },
    {
        "name": "get_current_datetime",
        "description":"現在の日時を取得する",
        "parameters": {
            "type":"object",
            "properties": {
                "now_time": {
                    "type": "string",
                    "description":"現在日時",
                }
            },
            "required":["now_time"],
        },
    },

]

In [189]:
def run_conversation(content, initial_messages=[]):
# ステップ1: ユーザー入力と関数の定義を GPT に送る
    user_message = {"role": "user", "content": content}
    messages = initial_messages + [user_message]
    print("ステップ1:", "ユーザー入力: ", user_message, ", 関数の定義: ", functions)
    # chatCompletionsを呼び出すための呼び出し方に注意
    # https://platform.openai.com/docs/guides/text-generation/chat-completions-api
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        # auto にすると、使うべき関数を自動で判定してくれる
        # もちろん入力内容によっては関数を使わなくてよいと判定されることもある
        function_call="auto",
    )
    # return response
    response_message = response.choices[0].message
    messages.append(response_message)
    print("原型のデータ:", response)
    print("ステップ1.5:", response.choices[0].message.content)
    print("ステップ2:", response_message)
    # ステップ2: GPT が関数を呼ぶべきか判定したかどうか確認する
    print("function_callingの利用確認:", response_message.function_call)
    # print(response_message.function_call.name) # ここに利用したfunction_callingの関数が入るが、ない場合エラーになるので割愛
    # print("必要情報の取得:", json.loads(response_message.function_call.arguments))
    if response_message.function_call:
        function_name = response_message.function_call.name
        function_args = json.loads(response_message.function_call.arguments)
        print(function_args)
        if function_name == "get_current_weather":
            # ステップ3: 関数を実行する
            city_name = function_args['city_name']
            function_response = get_current_weather(city_name)
            print("ステップ3:", function_response)
        elif function_name == "get_current_datetime":
            current_time = function_args['now_time']
            function_response = get_current_datetime()
            print("ステップ3:", function_response)
            
        function_result_message = {
            # role は function にすることに注意
            "role": "function",
            "name": function_name,
            # JSON を文字列に変換したときに日本語が \u6771 のように Unicode になってしまうため、ensure_ascii=False にして回避する
            "content": json.dumps(function_response, ensure_ascii=False),
        }
        # ステップ4: 実行した関数の名前と結果を GPT に送る
        messages.append(function_result_message)
        print(messages)
        second_response = openai.chat.completions.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )
        print("ステップ4:", function_result_message)
        # ステップ5: GPT からの回答を得る
        print("ステップ5:", second_response.choices[0].message.content)
    return messages


In [191]:
res = run_conversation("東京の天気を教えてください") # 天気予報を取得する関数を選択して解答を返してくれる
# res2 = run_conversation("現在の日時を教えてください") # 時間を取得する関数を選択して解答を返してくれる
# res3 = run_conversation("AIについてプログラミングするためにおすすめの言語を教えてください") # どの関数にも該当しないプロンプトなので、通常のモデルで解答をしてくれる

ステップ1: ユーザー入力:  {'role': 'user', 'content': '東京の天気を教えてください'} , 関数の定義:  [{'name': 'get_current_weather', 'description': '指定された都市の現在の天気を取得する', 'parameters': {'type': 'object', 'properties': {'city_name': {'type': 'string', 'description': '英語表記の都市名'}}, 'required': ['city_name']}}, {'name': 'get_current_datetime', 'description': '現在の日時を取得する', 'parameters': {'type': 'object', 'properties': {'now_time': {'type': 'string', 'description': '現在日時'}}, 'required': ['now_time']}}]
原型のデータ: ChatCompletion(id='chatcmpl-8tEGmdoGRzWqUQNOvfs6kN2hhXIS4', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "city_name": "Tokyo"\n}', name='get_current_weather'), tool_calls=None))], created=1708173584, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=123, total_tokens=141))
ステップ1.5: None
ステップ2: ChatCo

In [51]:
# print(type(res))
# print(vars(res))
# res_dict = res.__dict_
test = res.choices[0].message
print(test)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "city_name": "Tokyo"\n}', name='get_current_weather'), tool_calls=None)


### functioncalling 並列実行
上記までのfunciton callingは1レスポンスにつき1関数を1回しか実行できなかったそうだが、新しいfunction callingではfunctionsを並列で実行できるようになったらしいので実装してみる

In [231]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "東京", "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "サンフランシスコ", "temperature": "72", "unit": "fahrenheit"})
    elif "paris" in location.lower():
        return json.dumps({"location": "パリ", "temperature": "22", "unit": "celsius"})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})
    

tools = [
        # {
        #     # typeという指定が増えてネストが1段階深くなっている　typeで指定できるものが増えることで何かが増えるのか？
        #     "type": "function",
        #     "function": {
        #         "name": "get_current_weather",
        #         "description": "指定された都市の現在の天気を取得する",
        #     },
        #     "parameters": {
        #         "type": "object",
        #         "properties": {
        #             "city_name": {
        #                 "type": "string",
        #                 "description": "city_name:英語表記の都市名",
        #             },
        #         },
        #     },
        #     "required": ["city_name"],
        # },
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
        
]

In [235]:
def multi_run_cunversation(content, initial_message=[]):
    # ステップ1：ユーザ入力と関数の定義をGPTに送信
    user_message = {"role": "user", "content":content}
    messages = initial_message + [user_message]
    print("ステップ1:", "ユーザ入力：", user_message, "関数の定義:",tools )
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-1106", # 新しめのモデルでないと使えないらしいです
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    # ステップ2 GPTが関数を呼ぶべきかどうかを判定している
    print("ステップ2",response.model_dump_json(indent=2))
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # ステップ3：関数を実行すべきだった場合
    if tool_calls:
        messages.append(response_message)
        available_functions = {
            "get_current_datetime": get_current_datetime,
            "get_current_weather": get_current_weather,
        }

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            print("function_name:", function_name)
            print("function_to_call:", function_to_call)
            print("function_args:", function_args)
            
            # function_response = function_to_call(city_name=function_args.get("city"))
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
                }
            )
            print(messages)
        
        second_response  =openai.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )
        print(second_response.choices[0].message.content)
        return second_response
            # function_args = json.loads(tool_call.function.arguments)
            # if function_name == "get_current_weather":
            #     city_name = function_args["city"]
            #     function_response = get_current_weather(city_name)
            #     print("function_args:", function_args)
            #     print("ステップ3:",function_response)
            # elif function_name ==  "get_current_datetime":
            #     function_response = get_current_datetime()
            #     print("ステップ3:",function_response)
            # else:
            #     raise Exception
            # # それぞれの関数の実行結果をまとめる
            # messages.append(
            #         {
            #             "tool_call_id": tool_call.id,
            #             "role": "tool",
            #             "name": function_name,
            #             "content": function_response,
            #         }
            #     )
            
            # # 呼ぶべき関数の実行結果と初回リクエストとレスポンスを1つのリストにまとめる

            # print("ステップ3.5:",messages)
            # print(messages[1])
            # print(messages[2])
            
        # # ステップ4　準備した情報をもとに再度 LLMに問い合わせ
        # second_response = openai.chat.completions.create(
        #     model="gpt-3.5-turbo-1106", # 新しめのモデルでないと使えないらしいです
        #     messages=messages,
        # )
        # print("ステップ4", second_response.choices[0].message.content)
        # return second_response
            # 関数を実行する(ここのコードは汚いので直せるならなおす)
            # if function_name in available_functions:
            #     function_arguments = function_call.arguments
            #     print(function_arguments)
            #     function_response = eval(function_name)(**eval(function_arguments))
            #     print(function_response)

In [236]:
test = multi_run_cunversation("東京とサンフランシスコの気温を教えて")

ステップ1: ユーザ入力： {'role': 'user', 'content': '東京とサンフランシスコの気温を教えて'} 関数の定義: [{'type': 'function', 'function': {'name': 'get_current_weather', 'description': '指定された都市の現在の天気を取得する'}, 'parameters': {'type': 'object', 'properties': {'city_name': {'type': 'string', 'description': 'city_name:英語表記の都市名'}}}, 'required': ['city_name']}, {'type': 'function', 'function': {'name': 'get_current_datetime', 'description': '現在の日時の取得をする'}, 'parameters': {'type': 'object', 'properties': {'now_time': {'type': 'string', 'description': '現在日時'}}}, 'required': ['now_time']}]
ステップ2 {
  "id": "chatcmpl-8tFAcI9la6NrQ2ePi4zji5vkaepdW",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "role": "assistant",
        "function_call": null,
        "tool_calls": [
          {
            "id": "call_12xSFb4V1rlgh8kHA3DK3jbZ",
            "function": {
              "arguments": "{\"city\": \"Tokyo\"}",
              "name":

TypeError: get_current_weather() got an unexpected keyword argument 'location'

### LangChainのAgentでFunction callingを実装